In [1]:
import word_2_index as w2i
from mercari_config import MercariConfig

In [2]:
word2index = w2i.load_word2index(MercariConfig.WORD_2_INDEX_4_NAME_FILE)

In [3]:
len(word2index)

129912

In [4]:
word2index.count()

word_id        129912
entity_type    129912
count          129912
dtype: int64

In [5]:
word2index.groupby("entity_type").count()

,word_id,count
entity_type,,
CARDINAL,2495,2495
DATE,1144,1144
EVENT,377,377
FAC,662,662
GPE,2026,2026
LANGUAGE,1,1
LAW,126,126
LOC,429,429
MONEY,225,225


In [6]:
word2index[(word2index.word_cnt >= 10) & 
           (word2index.entity_type != MercariConfig.NON_ENTITY_TYPE)].sort_values('word_cnt', ascending=False)

,word_id,entity_type,cnt
word,,,
2,2176,CARDINAL,5070
Victoria,105144,GPE,4201
3,3246,CARDINAL,3626
iPhone,118597,LOC,3584
VS,103414,PRODUCT,3395
Nike,73691,ORG,2806
4,4120,CARDINAL,2627
NWT,69831,ORG,2519
Lularoe,59061,PERSON,1811


In [7]:
word2index[(word2index.word_word_cnt >= 2)].sort_values('word_cnt', ascending=False)

,word_id,entity_type,cnt
word,,,
's,225,___NONE_ENTITY___,10779
!,8,___NONE_ENTITY___,10352
size,125139,___NONE_ENTITY___,6741
for,117103,___NONE_ENTITY___,6540
-,247,___NONE_ENTITY___,6321
and,112223,___NONE_ENTITY___,6206
Bundle,21412,___NONE_ENTITY___,5208
2,2176,CARDINAL,5070
PINK,78143,___NONE_ENTITY___,5059
